In [10]:
from pynq.overlays.base import BaseOverlay
from pynq.lib import Pmod_ADC
base = BaseOverlay("base.bit")
import time
import random
import socket
import asyncio

In [11]:
adc = Pmod_ADC(base.PMODA)
btns = base.btns_gpio

In [18]:
def connect_to_server(HOST, PORT):
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    sock.connect((HOST, PORT))
    
    return sock


In [24]:
async def get_btns(loop,PORT):
    client_sock = None
    print('Starting the client code')
    time.sleep(1)
    while True:
        await asyncio.sleep(0.01)
        
        #button 3,2,1 stops the code
        if btns[3].read() != 0:
            break
            if client_sock: 
                client_sock.sendall(b'close')
                print('Ending Session and breaking')
                client_sock.close()
                loop.close() 
                break
        #end session
        elif btns[2].read() != 0:
            if client_sock:
                client_sock.sendall(b'close')
                print('Ending Session and breaking')
                client_sock.close()
                loop.close() 
                break
        # Send signal to buzz other pynq
        elif btns[1].read() != 0:
            print('pressed btn 1')
            if client_sock: 
                client_sock.sendall(b'close')
                print('Ending Session and breaking')
                client_sock.close()
                loop.close() 
                break
        # Connect to server button
        elif btns[0].read() != 0:
            if client_sock == None:
                try:
                    client_sock = connect_to_server('192.168.2.1',PORT)
                    #client_sock = connect_to_server('192.168.2.99',1000)
                    print('Connected to Server')
                    await asyncio.sleep(1)
                except:
                    print('Unable to connected to Server')
                    await asyncio.sleep(0.3)
            else: 
                print('Already connected')
                
        if client_sock:
            client_sock.sendall(str((adc.read_raw(ch1=1, ch2=1, ch3=0))).encode('ASCII'))
            await asyncio.sleep(4)

In [20]:
def run_client(PORT):
    loop = asyncio.new_event_loop()
    loop.create_task(get_btns(loop, PORT))
    loop.run_forever()
    loop.close() 

In [26]:
run_client(20002)

Starting the client code


KeyboardInterrupt: 

In [ ]:
procs = [] # a future list of all our processes

print('starting p1')
# Launch process1 on CPU0
p1 = multiprocessing.Process(target=run_client)
os.system("taskset -p -c {} {}".format(1, p1.pid)) # taskset is an os command to pin the process to a specific CPU
procs.append(p1)

print('starting p2')
# Launch process1 on CPU0
p2 = multiprocessing.Process(target=server, args=(1001,))
os.system("taskset -p -c {} {}".format(0, p2.pid)) # taskset is an os command to pin the process to a specific CPU
procs.append(p2)

print('Start')
p1.start() # start the process
p2.start() # start the proces

p1Name = p1.name # get process1 name
p2Name = p2.name # get process1 name
                             
p1.join()
p2.join()

print('Process 1 with name, {}, is finished'.format(p1Name))
print('Process 2 with name, {}, is finished'.format(p2Name))